In [1]:
import sys
import os
import time
from tqdm.notebook import tqdm
from time import sleep
sys.path.append('..')

In [2]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [3]:
from awsdet.utils.misc.config import Config
from awsdet.utils.logger import get_root_logger
from awsdet.utils.runner.dist_utils import init_dist
from awsdet.datasets.data_generator import DataGenerator
from awsdet.datasets import build_dataset
from awsdet.datasets.loader.build_loader import build_dataloader

import matplotlib.pyplot as plt
from awsdet.models.builder import build_backbone, build_neck, build_head

from awsdet.datasets.data_generator import DataGenerator

In [4]:
#config_path = '/workspace/shared_workspace/deep-learning-models/configs/rpn/rpn_r50_fpn_1x_coco.py'
config_path = '../configs/faster_rcnn/EC2/faster_rcnn_r50_fpn_1x_coco.py'
cfg = Config.fromfile(config_path)
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
cfg.work_dir = os.path.join('./work_dirs',
                                os.path.splitext(os.path.basename(config_path))[0])
log_file = os.path.join(cfg.work_dir, f'{timestamp}.log')

In [5]:
init_dist()
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [6]:
coco = build_dataset(cfg.data.train)

loading annotations into memory...
Done (t=16.20s)
creating index...
index created!


In [7]:
train_tdf, train_size = build_dataloader(coco, 4)
train_tdf_iter = iter(train_tdf.prefetch(32).repeat())

In [8]:
imgs, img_metas, gt_bboxes, gt_labels = next(train_tdf_iter)
#imgs = tf.cast(imgs, tf.float16)
#img_metas = tf.cast(img_metas, tf.float16)
#gt_bboxes = tf.cast(gt_bboxes, tf.float16)


Starting new loop for GPU: 0


In [9]:
backbone = build_backbone(cfg.model.backbone)
neck = build_neck(cfg.model.neck)
rpn_head = build_head(cfg.model.rpn_head)
backbone.layers[0].load_weights(cfg.model.backbone.weights_path)

In [10]:
@tf.function
def forward(imgs):
    training = True
    C2, C3, C4, C5 = backbone(imgs, training=training)
    P2, P3, P4, P5, P6 = neck((C2, C3, C4, C5), training=training)
    rpn_feature_maps = [P2, P3, P4, P5, P6]
    '''rpn_class_logits, rpn_probs, rpn_deltas = rpn_head(rpn_feature_maps, training=training)'''
    return rpn_feature_maps

In [12]:
%%timeit
imgs, img_metas, gt_bboxes, gt_labels = next(train_tdf_iter)
#imgs = tf.cast(imgs, tf.float16)
rpn_feature_maps = forward(imgs)

1.28 s ± 3.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
feature_maps_sizes = [tf.shape(i)[1:3] for i in rpn_class_logits]
anchor_list, valid_flag_list = rpn_head.get_anchors(feature_maps_sizes, img_metas)
num_imgs = len(img_metas)

In [24]:
flat_anchors = tf.concat(anchor_list[0], axis=0)
flat_flags = tf.concat(valid_flag_list[0], axis=0)

In [21]:
anchor_list

[[<tf.Tensor: shape=(334668, 4), dtype=float32, numpy=
  array([[ -22.627417,  -11.313708,   22.627417,   11.313708],
         [ -16.      ,  -16.      ,   16.      ,   16.      ],
         [ -11.313708,  -22.627417,   11.313708,   22.627417],
         ...,
         [1309.3726  , 1320.6863  , 1354.6274  , 1343.3137  ],
         [1316.      , 1316.      , 1348.      , 1348.      ],
         [1320.6863  , 1309.3726  , 1343.3137  , 1354.6274  ]],
        dtype=float32)>,
  <tf.Tensor: shape=(83667, 4), dtype=float32, numpy=
  array([[ -45.254833,  -22.627417,   45.254833,   22.627417],
         [ -32.      ,  -32.      ,   32.      ,   32.      ],
         [ -22.627417,  -45.254833,   22.627417,   45.254833],
         ...,
         [1282.7451  , 1305.3726  , 1373.2549  , 1350.6274  ],
         [1296.      , 1296.      , 1360.      , 1360.      ],
         [1305.3726  , 1282.7451  , 1350.6274  , 1373.2549  ]],
        dtype=float32)>,
  <tf.Tensor: shape=(21168, 4), dtype=float32, numpy=
 